# Introducción a Dask

Dask es una biblioteca de python diseñada para cómputo científico/analítico en gran escala. Existen dos grandes componentes de  Dask:

1. **Cómputo de tareas**: Para ejecutar un _workflow_ o _pipeline_ de tareas en paralelo. Esto es similar a Airflow o Luigi, pero de "bajo nivel".
2. **Grandes datos**: Datos que no quepan en memoria o no se puedan procesar cómodamente en una computadora personal. Dask provee una solución similar a Spark, pero con sintáxis de pandas/numpy. Este componente está construido sobre el anterior

## ¿Por qué Dask?

![](resources/growth.png)

* API Familiar (parecido a numpy/pandas)
* Python no es un extra 
* Escala fácilmente y de manera flexible
* Desarrollo constante 
* Comunidad activa

## Dask vs Spark

### Lenguaje
* Spark está escrito en Scala con soporte (secundario) para Python y R. Puede utilizar código para la JVM
* Dask está escrito en Python y sólo soporta Python. Puede utilizar código de C/C++/Fortran a través de las integraciones con Python

### Ecosistema
* Spark es parte de un ecosistema inspirado en el (Sparkling Water, Spark ML, etc). "Juega bonito" con otros proyectos de Apache
* Dask es parte del ecosistema Python. "Juega bonito" con otras librerías de Python como pandas, numpy, etc

### Edad y Confianza
* Spark es más viejo y maduro (2010), y por lo tanto se ha ganado un lugar en la industria
* Dask es más joven (2014) y en desarrollo constante, pero intenta ser una extensión de otras librerías confiables como numpy/pandas

### Alcance
* Spark está diseñado para trabajar con abstracciones de alto nivel. Tareas típicas de BI y queries SQL son sus principales casos de uso
* Dask está pensado para trabajar a un nivel más bajo. Paralelizar y ejecutar tareas arbitrarias es uno de sus objetivos, aunque también incluye algunas abstracciones de alto nivel.

### Escoge Spark si...
* Prefieres Scala o SQL
* Necesitas operar con código basado en la JVM
* Quieres una solución establecida y robusta para producción
* Necesitas programar ETLs sencillos
* Principalmente realizas tareas de BI con un poco de ML

### Escoge Dask si...
* Prefieres Python o código nativo (C/Fortran)
* Tu caso de uso es complejo y no es fácil de manejar en Spark
* Prefieres un proyecto más ligero que te permita realizar cómputo en paralelo


## Conéctandonos al cluster

In [ ]:
from dask.distributed import Client

client = Client()
client

La manera de distribuir trabajo en Dask es similar a Spark. Tenemos un sólo _scheduler_ que se encarga de agendar las tareas y distribuirlas a los _workers_ activos. Esto nos permite escalar a cientos de CPUs.

![](resources/workers.png)

Dask también nos permite monitorear recursos del sistema

![](resources/system.png)

También nos permite monitorear la ejecución de tareas

![](resources/task_stream.png)

Al igual que en Spark, podemos crear un cliente dirigiéndonos simplemente a la dirección del _scheduler_

## Computo _Lazy_ y en paralelo



Con Dask podemos generar una "receta" para la ejecución de tareas, sin ejecutarla en ese momento. Esto nos permite no "bloquear" nuestro programa y es lo que permite la paralelización

In [ ]:
from dask import delayed

# Decoramos nuestras funciones con `delayed`, para indicar que no se deben ejecutar en este momento

@delayed
def inc1(x):
    return x + 1


@delayed
def mul2(x):
    return x*2


@delayed
def add(x, y):
    return x + y

# Creamos la "receta" de ejecución
x = inc1(15)
y = mul2(7)
total = add(x, y)
total

In [ ]:
# Podemos visualizar el grafo de tareas generado por esta receta
total.visualize()

In [ ]:
# Realizamos el cómputo de la tarea
result = total.compute()
print(result)

# O bien, lo realizamos de manera remota 
computed = client.compute(total) # Esto no bloquea
computed.result()

In [ ]:
import numpy as np

def sequential(s):
    a = np.random.normal(0, 1, s)
    b = np.random.normal(5, 3, s)
    c = np.random.normal(7, 2, s)
    
    return np.percentile(a, 80) + np.percentile(b, 60) + np.percentile(c, 99)

delayed_normal = delayed(np.random.normal)
delayed_percentile = delayed(np.percentile)
def in_parallel(s):
    a = delayed_normal(0, 1, s)
    b = delayed_normal(5, 3, s)
    c = delayed_normal(7, 2, s)

    return (delayed_percentile(a, 80) + delayed_percentile(b, 60) + delayed_percentile(c, 99)).compute()

print("Sequential:")
%time sequential(int(1e7))

print("Parallel:")
%time in_parallel(int(1e7))

In [ ]:
params = [(0, 1), (5, 3), (7, 2)]
percentiles = [80, 60, 99]
def in_parallel(s, params, percentiles):
    normal_samples = [delayed_normal(p[0], p[1], s) for p in params]
    return sum([delayed_percentile(sample, p) for sample, p in zip(normal_samples, percentiles)]).compute()
    
%time in_parallel(int(1e7), params, percentiles)

* ¿Cómo cambia este resultado con el parámetro de tamaño `s`? ¿Por qué?
* ¿Cómo cambiarías la función para usar ciclos y parámetros (de la normal y percentiles)?

In [ ]:
# Ejercicio: escribe una función para checar si un número es primo y pruébala con los sig. numeros. 
#  Después, ejecuta el ciclo en paralelo y compara el tiempo de ejecución de ambos métodos
import math
nums = [7961147, 1627785787, 402938420121, 2093482093844387, 
        211236347099, 3718687359123, 871144421117, 650280751121]

def is_prime(n):
    '''Checa si n es primo o no'''
    # Hint: sólo tienes que checar los factores hasta la raíz cuadrada de n
    return False

%time [is_prime(n) for n in nums]

delayed_is_prime = delayed(is_prime)
%time [delayed_is_prime(n).compute() for n in nums]

## _Bags_

Los _bags_ de Dask están diseñados para lidiar con datos semi-estructurados o no estructurados (e.g. documentos JSON, texto plano). Son análogos a los RDD de Spark y proveen una abstracción a los conceptos de `map`, `filter`, etc.

In [ ]:
from dask import bag

taxis = bag.read_text("../data/taxis.json")

# Cada línea es un JSON en texto
print(taxis.take(1))

In [ ]:
import json
taxis = taxis.map(json.loads) # Lazy!
taxis.take(1) #Sólo ejecuta un `json.loads`

In [ ]:
# Filter
b_type = taxis.filter(lambda d: d["car_type"] == "B")
print("Autos tipo B: {}".format(b_type.count().compute()))

In [ ]:
# Pluck
print("Primeros 10 ids: {}".format(taxis.pluck("taxi_id").take(10)))

In [ ]:
# Aplasta
trips = taxis.pluck("trips").flatten()
trips.take(5)

In [ ]:
# Algunas operaciones se pueden aplicar directamente
(trips.pluck("fare_amount")
      .mean()
      .compute())

### Group By
Agrupa los elementos del bag de acuerdo a la evaluación de alguna función "llave"

In [ ]:
b = bag.from_sequence(list(range(10)))
b.groupby(lambda x: x % 3).compute()

### Fold By

Agrupa y **reduce** de acuerdo a la evaluación de alguna función. Cumple un propósito similar a `reduceByKey` o `combineByKey` de Spark.

Necesita 3 cosas:

1. Una función "llave" para agrupar los elementos
2. Una función binaria para reducir elementos por pares
3. Una función binaria para reducir el resultado de dos reducciones

In [ ]:
%%time

def incr(x, y):
    return x + 1

def add(x, y):
    return x + y

print(trips.foldby(key=lambda x: x["tip_amount"] > 10, # Si el tip es > 10 o no
              binop=incr, # Incrementa 1 por obs
              initial=0, 
              combine=add, # Suma cada cuenta
              combine_initial=0)
       .compute())

In [ ]:
%%time

(trips.groupby(grouper=lambda x: x["tip_amount"] > 10, shuffle="disk")
      .starmap(lambda k, v: (k, len(v)))
      .compute())

¿Por qué es tan lento `groupby` comparado con `foldby`?

In [ ]:
## Ejercicio: Denormalización
# Obtener un dataframe de trips a partir de nuestro bag de taxis SIN PERDER la información del taxi
# Hint: Una vez que el bag contenga los elementos correctos, usar `my_bag.to_dataframe()`



## _Dataframes_

El dataframe es un concepto popular en análisis de datos para manejar datos estructurados de manera tabular. Dask implementa una interface (API) de dataframes similar a la de pandas. De hecho, un dataframe de Dask está formado de varios dataframes de pandas en paralelo, y una operación en el dataframe de Dask genera una operación para cada uno de esos dataframes.

In [ ]:
from dask import dataframe
trips_df = dataframe.read_csv("../data/trips.csv")
trips_df.head()

In [ ]:
# Truco para convertir a datetime...
trips_df.tpep_pickup_datetime = trips_df.tpep_pickup_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime = trips_df.tpep_dropoff_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime.head()

En general, los dataframes de pandas implementan más funciones (i.e. tienen más features) que los de Dask. Los dataframes de Dask son una manera de interactuar _a la pandas_ con datos que no caben en memoria

### Ejercicios simples

In [ ]:
# Usa `describe` para obtener información básica del dataframe

In [ ]:
# Calcula la propina promedio para cada tipo de coche (A, B)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Grafica dos boxplot para comparar la distribución de propina según el tipo de coche

In [ ]:
# Grafica un histograma de los viajes según la hora del día

In [ ]:
# Agrega una columna para la duración del viaje
trips_df["trip_duration"] = trips_df.tpep_dropoff_datetime - trips_df.tpep_pickup_datetime
trips_df.trip_duration.map(lambda d: d.total_seconds()).head()

In [ ]:
# Grafica un scatterplot de duración de viaje vs. distancia de viaje

# Para un scatterplot de verdad la única manera es traer los datos completos, o usar un sample
print(len(trips_df))
local_sample = trips_df.sample(0.1).compute()

plt.scatter(local_sample.trip_duration.map(lambda d: d.total_seconds()), local_sample.trip_distance)

In [ ]:
# ¿Cuál es la proporción de propina promedio para los viajes con más de 3 pasajeros?
trips_df[trips_df.passenger_count > 3].tip_amount.mean().compute()

## Indices

Dada la naturaleza en paralelo de Dask, los índices cobran mucho mayor relevancia. Todas las operaciones que puedan hacerse sobre un índice serán mucho más rápidas, pero cambiar de índice será costoso

In [ ]:
trips_indexed = trips_df.set_index("tpep_pickup_datetime") # Muy costoso en datos grandes, evitar!
trips_indexed.divisions

In [ ]:
import pandas as pd
trips_indexed.loc[pd.Timestamp('2015-01-01 01:08:55')].head()

## Ejercicio/Tarea

Aprovecha la capacidad de Dask para realizar cómputo en paralelo para ajustar un modelo para predecir la proporción de propina de un viaje. Realiza búsqueda de hiperparámetros en grid con cross validation. Puedes usar funciones de scikit learn. Recuerda usar el decorador `delayed` para ejecutar en paralelo.

* ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

Haz lo mismo que arriba, pero utilizando la biblioteca Dask-ML http://dask-ml.readthedocs.io/en/latest/ 

* ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?

**Bonus**

Haz lo mismo utilizando Spark ML

* ¿Cómo se comparan los tiempos de ejecución de Spark vs Dask?

Usa los datos en s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv

* ¿Cambia alguno de los resultados anteriores?